# LangChain:输出封装OutputParser

## basic

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser

prompt1='说出几种常见的水果'
prompt2="输出20以内的素数"
llm=ChatOpenAI()
comma_parser=CommaSeparatedListOutputParser()
response=llm.invoke(prompt1)
print("===模型原始输出===")
print(response.content)
print('===CommaSeparatedListOutputParser解析输出===')
print(comma_parser.parse(response.content))
response=llm.invoke(prompt2)
print("===模型原始输出===")
print(response.content)
print('===CommaSeparatedListOutputParser解析输出===')
print(comma_parser.parse(response.content))


Note: 针对prompt1模型输出的结果中的没有,所以CommaSeparatedListOutputParser解析器没有正确解析出来。

## 1.1 langchain_core中的解析器

### 目前langchain_core.output_parsers中的OutputParser类主要包括以下几种(虚拟类不列出)：

* json类：JsonOutputParser，将结果解析成json串。
* list类： CommaSeparatedListOutputParser、MarkdownListOutputParser和NumberedListOutputParser，将结果解析成类似List的对象。CommaSeparatedListOutputParser其结果是对LLM模型返回的字符串使用(,)进行分割得到的List。后面两种分别采用正则表达式从LLM模型返回的字符串中提取出符合规则的部分。
* string类。StrOutputParser，直接输出LLM模型返回的结果。
* xml类。XMLOutputParser，将LLM模型返回的结果解析成xml。
* pydantic类。PydanticOutputParser, 第2部分介绍。
* openai_function和openai_tools类：这两类可以帮助提取出LLM产生的函数调用的参数。openai_function系列针对的是使用function_call字段表示函数调用的参数的情况，后者针对的使用tool_calls字段表示function call的参数的情况(GPT系列模型采用的是这种方式)。这里只介绍openai_tools类。主要包括JsonOutputKeyToolsParser、JsonOutputToolsParser和PydanticToolsParser。其用法举例如下：

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.outputs import ChatGeneration
#注意引用方法
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
tools=[{ 
            "type": "function",
            "function": {
                "name": "sum",
                "description": "加法器，计算一组数的和",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "numbers": {"type": "array", "items": { "type": "number"}}
                    }
                }
            }
        }
    ]

llm=ChatOpenAI(model_kwargs={"tools":tools})

response=llm.invoke("计算一组数据的和:13,49837,3489,23423")
print(response)
response=ChatGeneration(message=response)

parser=JsonOutputToolsParser()
print("---解析后结果---")
print(parser.parse_result([response]))


## 1.2 langchain中的输出封装
* boolean: BooleanOutputParser	返回True或False
* combining: CombiningOutputParser	同时将多个输出解析成同一个格式
* datetime: DatetimeOutputParser	将日期转化成特定格式
* enum: EnumOutputParser	将LLM模型输出解析成enum
* fix: OutputFixingParser	可以自动修复异常并重新解析
* pandas_dataframe: PandasDataFrameOutputParser	使用pandas.dataframe的format对LLM进行解析
* regex: RegexParser	使用正则表达式对LLM模型返回进行解析
* regex_dict: RegexDictParser	功能同上
* retry: RetryOutputParser、RetryWithErrorOutputParser	可以自动修复异常并重新解析
* structured	StructuredOutputParser	将LLM模型输出进行结构化解析
* yaml	YamlOutputParser	提取LLM模型输出中的YAML部分

虽然OutputParser可以对LLM的输出结果进行解析，但这种使用方式用户体验并不好，正如第一个案例中针对prompt1的输出结果，因为无法确定LLM模型输出的结果中是否使用,，所以解析结果无法掌控。可以使用OutputParser类的get_format_instructions()方法来调整LLM模型输出。举例如下：

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

parser=DatetimeOutputParser()
format_instruction=parser.get_format_instructions()
llm=ChatOpenAI()
prompt=PromptTemplate.from_template(template="计算机时间的起始时间是什么时候？{format_instruction}")
prompt=prompt.format(format_instruction=format_instruction)
response=llm.invoke(prompt)
print(response.content)

In [ ]:
# from langchain.output_parsers import JsonOutputKeyToolsParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

parser = JsonOutputParser()
format_instruction = parser.get_format_instructions()
llm=ChatOpenAI(model="gpt-3.5-turbo")
prompt=PromptTemplate.from_template(template="计算机时间的起始时间是什么时候？{format_instruction}")
prompt=prompt.format(format_instruction=format_instruction)
response=llm.invoke(prompt)
print(response.content)

In [ ]:
from pydantic import BaseModel,Field,validator
import re

class Func_Json(BaseModel):
    name:str=Field(description="函数名称，只能由字母组成并且长度不超过10")
    description:str=Field(description="函数描述")

    @validator("name")
    def valid_name(cls,field):
        if re.match(r'^[a-zA-Z]+$',field) and len(field)<10:
            return field
        else:
            raise ValueError("函数名称不符合要求")

print(Func_Json(name="sum",description="求一组数据的和"))
print(Func_Json(name='sum2',description="求一组数据的和"))


In [ ]:
from pydantic import BaseModel,Field,validator
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from typing import List
from langchain_core.prompts import PromptTemplate

class New_List(BaseModel):
    item:List[str]

llm = ChatOpenAI()
parser = PydanticOutputParser(pydantic_object=New_List)

prompt = PromptTemplate.from_template("请列出10种常见的水果。{format_instruction}")
prompt = prompt.format(format_instruction = parser.get_format_instructions())

response = llm.invoke(prompt)
print(response.content)


### Sean test

In [ ]:
from pydantic import BaseModel,Field,validator, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from typing import List
from langchain_core.prompts import PromptTemplate
import json

class New_List(BaseModel):
    item1: List[str] = Field(description="name appears in the query")
    item2: List[str] = Field(description="terminology appears in the query")

llm = ChatOpenAI(model="gpt-3.5-turbo")

parser = PydanticOutputParser(pydantic_object=New_List)

prompt = PromptTemplate.from_template("20世紀初期海森堡和薛丁格發展出量子力學 {format_instruction}")
prompt = prompt.format(format_instruction = parser.get_format_instructions())

# print(parser.get_format_instructions())

response = llm.invoke(prompt)
# print(response)
print(response.content)

json_response = json.loads(response.content)
print(json_response['item1'])


In [ ]:
from pydantic import BaseModel,Field,validator, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from typing import List
from langchain_core.prompts import PromptTemplate
import json

class New_List(BaseModel):
    item1: List = Field(description="name appears in the query")
    item2: List = Field(description="terminology appears in the query")

llm = ChatOpenAI(model="gpt-3.5-turbo")

parser = PydanticOutputParser(pydantic_object=New_List)

# prompt = PromptTemplate.from_template("20世紀初期海森堡和薛丁格發展出量子力學 {format_instruction}")
prompt = PromptTemplate(
    template="{input} {format_instruction}",
    input_variables=["input"],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

# prompt = prompt.format(format_instruction = parser.get_format_instructions())
# print(parser.get_format_instructions())

chain = prompt | llm | parser

res = chain.invoke("20世紀初期海森堡和薛丁格發展出量子力學")
print(type(res))
print(res)

